In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#import sys
#!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# just set the seed for the random number generator
#np.random.seed(107)

In [ ]:
# Change...
pair = 'ETHBTC'

# interval used to compute indicators and backtest analysis
# valid strategy intervals - 1min, 3min, 5min, 15min, 30min, 1h, 2h, 4h, 6h, 8h, 12h, 1D, 3D, 1W, 1M
strategy_interval = '1h'

# date to start/end analysis
# Use '' for not to limit the data
#dt_start_analysis = '2021-09-09' # BTC
#dt_start_analysis = '2022-11-23' # CHZ
#dt_start_analysis = '2021-12-10' # MATIC
#dt_start_analysis = '2021-11-28' # ENJ
#dt_start_analysis = '2021-05-19' # ADA
#dt_start_analysis = '2022-06-08' # ETH
#dt_start_analysis = '2021-11-08' # TOP LAST CICLE
#dt_start_analysis = '2024-06-05'

# current year
#dt_start_analysis = date(date.today().year, 1, 1)
# current month
#dt_start_analysis = date(date.today().year, date.today().month, 1)
# months before
dt_start_analysis = date.today() - relativedelta(months=3)

#dt_end_analysis = '2022-11-22' # BOTTOM
#dt_end_analysis = '2021-11-08' # TOP LAST CICLE
dt_end_analysis = ''

periods = [2, 3, 5, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
# fibonacci
# periods = [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233]


In [ ]:
from binance_utils import init
from trade_utils import get_data

client = init()
data = get_data(client, pair, strategy_interval)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data.sample(10)

In [ ]:
data.info()

In [ ]:
from technical_indicator_utils import get_sma, get_ema, get_macd, get_rsi, get_adx, get_bbands, get_rvi, get_momentum, get_atr
from trade_utils import get_num_daily_bars

daily_bars = get_num_daily_bars(data)

# Momentum
data['Momentum'] = get_sma(get_momentum(data['ClosePrice'], daily_bars*14), 14)

# Calculate returns
data['Returns'] = data['ClosePrice'].pct_change() * 100 # pct_change() = percent change from previous price
data['Returns5'] = data['ClosePrice'].pct_change(5) * 100 # change from 5 previous price
data['DailyReturns'] = data['ClosePrice'].pct_change(daily_bars) * 100 # daily change
data['MonthlyReturns'] = data['ClosePrice'].pct_change(daily_bars*30) * 100 # monthly change

# Volatility
data['MonthlyVolatility'] = data.rolling(daily_bars*30, min_periods = daily_bars*30)['Returns'].std()

for period in periods:
    # SMA
    sma_name = 'SMA' + str(period)
    data[sma_name] = get_sma(data['ClosePrice'], period)

    # EMA
    ema_name = 'EMA' + str(period)
    data[ema_name] = get_ema(data['ClosePrice'], period)

# MACD
data['MACD'], data['MACDSignal'], data['MACDHist'] = get_macd(data['ClosePrice'])

# RVI
data['RVI'], data['RVISignal'] = get_rvi(data['OpenPrice'], data['ClosePrice'], data['LowPrice'], data['HighPrice'])

# RSI
data['RSI'] = get_rsi(data.ClosePrice)
data['RSI_10'] = get_rsi(data.ClosePrice, period=10)

# ADX
data['DI+'], data['DI-'], data['ADX'] = get_adx(data['HighPrice'], data['LowPrice'], data['ClosePrice'])

# Bollinger Bands
data['UpperBBand'], data['MidiBBand'], data['LowerBBand'], data['BBW'] = get_bbands(data['ClosePrice'])

# ATR
data['ATR'] = get_atr(data['HighPrice'], data['LowPrice'], data['ClosePrice'])

In [ ]:
#limit data to plot
if dt_end_analysis == '' and dt_start_analysis != '':
    data = data[dt_start_analysis:]
elif dt_end_analysis != '' and dt_start_analysis == '':
    data = data[:dt_end_analysis]
elif dt_end_analysis != '' and dt_start_analysis != '':
    data = data[dt_start_analysis:dt_end_analysis]

In [ ]:
from technical_indicator_utils import get_fibonacci_retracement_levels

# Fibonacci retracement
fib_levels = get_fibonacci_retracement_levels(data['ClosePrice'].max(), data['ClosePrice'].min())

In [ ]:
# Plot the histogram
data['Returns'].hist(bins=100, color='red')
plt.ylabel('Frequency')
plt.xlabel('Return')
plt.show()

In [ ]:
#plt.style.use('fivethirtyeight')

In [ ]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01,
                    row_heights=[0.3, 0.1, 0.3, 0.15, 0.15])

#fig = go.Figure()

fig.add_trace(go.Candlestick(x=data.index,
              open=data['OpenPrice'],
              high=data['HighPrice'],
              low=data['LowPrice'],
              close=data['ClosePrice'],
              name='Price'))
fig.add_trace(go.Scatter(x=data.index,
                         y=data['SMA40'],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name='SMA 40'))
fig.add_trace(go.Scatter(x=data.index,
                         y=data['SMA80'],
                         opacity=0.7,
                         line=dict(color='orange', width=2),
                         name='SMA 80'))
fig.add_trace(go.Scatter(x=data.index,
                         y=data['EMA200'],
                         opacity=0.7,
                         line=dict(color='red', width=2),
                         name='EMA 200'))

# Plot volume on 2nd row
colors = ['green' if row['OpenPrice'] - row['ClosePrice'] >= 0
          else 'red' for index, row in data.iterrows()]
fig.add_trace(go.Bar(x=data.index,
                     y=data['Volume'],
                     marker_color=colors,
                     name='Volume'
                    ), row=2, col=1)

# Plot Bollinger Bands 3rd row
fig.add_trace(go.Scatter(x=data.index,
                         y=data['ClosePrice'],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name='Price'), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index,
                         y=data['UpperBBand'],
                         opacity=0.7,
                         line=dict(color='red', width=2),
                         name='Upper band'), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index,
                         y=data['MidiBBand'],
                         opacity=0.7,
                         line=dict(color='yellow', width=2),
                         name='Middle band'), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index,
                         y=data['LowerBBand'],
                         opacity=0.7,
                         line=dict(color='red', width=2),
                         name='Lower band'), row=3, col=1)

# Plot MACD on 4th row
colors = ['green' if val >= 0
          else 'red' for val in data['MACDHist']]
fig.add_trace(go.Bar(x=data.index,
                     y=data['MACDHist'],
                     marker_color=colors,
                     name='Diff'
                    ), row=4, col=1)
fig.add_trace(go.Scatter(x=data.index,
                         y=data['MACD'],
                         line=dict(color='blue', width=1),
                         name='MACD'
                        ), row=4, col=1)
fig.add_trace(go.Scatter(x=data.index,
                         y=data['MACDSignal'],
                         line=dict(color='red', width=1),
                         name='Signal'
                        ), row=4, col=1)

# Plot RSI on 5th row
fig.add_trace(go.Scatter(x=data.index,
                         y=data['RSI'],
                         line=dict(color='purple', width=2),
                         name='RSI'
                        ), row=5, col=1)
fig.add_hrect(y0=30, y1=70, line_width=0, fillcolor="purple", opacity=0.2, row=5, col=1)

# update y-axis label
fig.update_yaxes(title_text='Price - SMA [40, 80] - EMA [200]', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)
fig.update_yaxes(title_text='Bollinger Bands', row=3, col=1)
fig.update_yaxes(title_text='MACD', showgrid=False, row=4, col=1)
fig.update_yaxes(title_text='RSI', showgrid=False, range = [0,100], row=5, col=1)

fig.update_layout(title=pair + '@' + strategy_interval,
                  xaxis_rangeslider_visible=False,
                  showlegend=False,
                  width=1400, height=1200,
                  hovermode='x unified',
                  margin=go.layout.Margin(
                    l=20, #left margin
                    r=20, #right margin
                    b=20, #bottom margin
                    t=50  #top margin
                  ))
fig.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9) = plt.subplots(
    nrows=9, sharex=True, figsize=(21, 28), gridspec_kw = {'height_ratios':[3, 3, 3, 1, 1, 1, 1, 1, 1]})

# Price x SMA
ax1.plot(data.index, data['ClosePrice'], label='Close Price')
ax1.plot(data.index, data['SMA30'], label='SMA 30')
ax1.plot(data.index, data['SMA100'], label='SMA 100')
# Fibonacci retracement
ax1.axhspan(fib_levels['level1'], fib_levels['price_min'], alpha=0.4, color='lightsalmon')
ax1.axhspan(fib_levels['level2'], fib_levels['level1'], alpha=0.5, color='palegoldenrod')
ax1.axhspan(fib_levels['level3'], fib_levels['level2'], alpha=0.5, color='palegreen')
ax1.axhspan(fib_levels['price_max'], fib_levels['level3'], alpha=0.5, color='powderblue')
ax1.legend()
ax1.set_ylabel('Price - SMA')

# Price x EMA
ax2.plot(data.index, data['ClosePrice'], label='Close Price')
ax2.plot(data.index, data['EMA20'], label='EMA 20')
ax2.plot(data.index, data['EMA90'], label='EMA 90')
ax2.plot(data.index, data['EMA200'], label='EMA 200')

# Fibonacci retracement
ax2.axhspan(fib_levels['level1'], fib_levels['price_min'], alpha=0.4, color='lightsalmon')
ax2.axhspan(fib_levels['level2'], fib_levels['level1'], alpha=0.5, color='palegoldenrod')
ax2.axhspan(fib_levels['level3'], fib_levels['level2'], alpha=0.5, color='palegreen')
ax2.axhspan(fib_levels['price_max'], fib_levels['level3'], alpha=0.5, color='powderblue')
ax2.legend()
ax2.set_ylabel('Price - EMA')

# Bollinger Bands
ax3.plot(data['ClosePrice'], label='Price')
ax3.plot(data['UpperBBand'], label='Upper band')
ax3.plot(data['MidiBBand'], label='Middle band')
ax3.plot(data['LowerBBand'], label='Lower band')
ax3.legend()
ax3.set_ylabel('Bollinger Bands')

# MACD
ax4.plot(data.index, data['MACD'], label='MACD', color = 'red')
ax4.bar(data.index, data['MACDHist'], label='Hist', color='purple', width = timedelta(hours=1))
ax4.plot(data.index, data['MACDSignal'], label='Signal', color='blue')
ax4.legend()
ax4.set_ylabel('MACD')

# RVI
ax5.plot(data.index, data['RVI'], label='RVI', color = 'red')
ax5.plot(data.index, data['RVISignal'], label='Signal', color='blue')
ax5.legend()
ax5.set_ylabel('RVI')

# RSI
ax6.plot(data.index, [70] * len(data.index), label='overbought')
ax6.plot(data.index, [30] * len(data.index), label='oversold')
ax6.plot(data.index, data['RSI'], label='RSI')
ax6.legend()
ax6.set_ylabel('RSI')

# DMI
ax7.plot(data.index, data[['DI+', 'DI-']])
ax7.plot(data.index, data[['ADX']], color='red')
ax7.plot(data.index, [25] * len(data.index), color = 'purple', linestyle='dashed')
ax7.set_ylabel('DMI')

# ATR
ax8.plot(data.index, data['ATR'], label='ATR')
ax8.legend()
ax8.set_ylabel('ATR')

# Volume
# make bar plots and color differently depending on up/down for the day
pos = data['OpenPrice'] - data['ClosePrice'] < 0
neg = data['OpenPrice'] - data['ClosePrice'] > 0
ax9.bar(data.index[pos], data[pos]['Volume'], color='green', width = timedelta(hours=1))
ax9.bar(data.index[neg], data[neg]['Volume'], color='red', width = timedelta(hours=1))

#ax7.bar(data.index, data['Volume'] / 1000000, width = timedelta(hours=4))
ax9.set_ylabel('Volume')

fig.tight_layout()

## Backtest Strategy

In [ ]:
import bt

# Get the price data
price_data = data[['ClosePrice']].copy()

### Golden/Death Cross strategy

In [ ]:
from strategy_utils import get_cross_signal
from backtest_utils import signal_strategy

#signal = get_cross_signal(data[['SMA50']].copy(), data[['SMA200']].copy())
signal = get_cross_signal(data[['SMA30']].copy(), data[['SMA70']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_sma_cross = signal_strategy(price_data.copy(), signal, 'sma_cross')

bt_result = bt.run(bt_sma_cross)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
bt_result.get_transactions()

### RSI strategy

In [ ]:
from strategy_utils import get_rsi_signal
from backtest_utils import signal_strategy

overbought_values = [60, 65, 70, 75, 80, 85]
oversold_values = [40, 35, 30, 25, 20, 15]

signal = None
bt_rsi = None
bt_result = None
overbought_value_rsi = 0
oversold_value_rsi = 0

# find best overbought and oversold value
total_return_best = None
for obv in overbought_values:
    for osv in oversold_values:
        strategy_name = 'rsi' + str(obv) + '_' + str(osv)

        signal_tmp = get_rsi_signal(data[['RSI']].copy(), overbought_value=obv, oversold_value=osv)
        bt_rsi_tmp = signal_strategy(price_data.copy(), signal_tmp, strategy_name)
        bt_result_tmp = bt.run(bt_rsi_tmp)

        print(strategy_name + ': ' + str(bt_result_tmp.stats.loc['total_return'].values[0]))

        if (total_return_best is None) or (bt_result_tmp.stats.loc['total_return'].values[0] > total_return_best):
            signal = signal_tmp
            bt_rsi = bt_rsi_tmp
            bt_result = bt_result_tmp
            overbought_value_rsi = obv
            oversold_value_rsi =  osv

            total_return_best = bt_result_tmp.stats.loc['total_return'].values[0]

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))
fig.suptitle('RSI ' + str(overbought_value_rsi) + '/' + str(oversold_value_rsi), fontsize=14)

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [overbought_value_rsi] * len(data.index), label='overbought')
ax2.plot(data.index, [oversold_value_rsi] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

### SMA Cross strategy

In [ ]:
from strategy_utils import get_cross_signal
from backtest_utils import signal_strategy

signal = None
bt_sma = None
bt_result = None
short_value = 0
long_value = 0

# find best short and long value
total_return_best = None
for short in periods:
    for long in periods:
        if (long > short):
            strategy_name = 'sma_cross' + str(short) + '_' + str(long)
            short_description = 'SMA' + str(short)
            long_description = 'SMA' + str(long)

            signal_tmp = get_cross_signal(data[[short_description]].copy(), data[[long_description]].copy())
            bt_sma_tmp = signal_strategy(price_data.copy(), signal_tmp, strategy_name)
            bt_result_tmp = bt.run(bt_sma_tmp)

            print(strategy_name + ': ' + str(bt_result_tmp.stats.loc['total_return'].values[0]))

            if (total_return_best is None) or (bt_result_tmp.stats.loc['total_return'].values[0] > total_return_best):
                signal = signal_tmp
                bt_sma = bt_sma_tmp
                bt_result = bt_result_tmp
                short_value = short
                long_value =  long

                total_return_best = bt_result_tmp.stats.loc['total_return'].values[0]

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(20, 8))
fig.suptitle('SMA Cross ' + str(short_value) + '/' + str(long_value), fontsize=16)

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(bt_result.prices)
ax2.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

### EMA Cross Strategy

In [ ]:
from strategy_utils import get_cross_signal
from backtest_utils import signal_strategy

signal = None
bt_ema = None
bt_result = None
short_value = 0
long_value = 0

# find best short and long value
total_return_best = None
for short in periods:
    for long in periods:
        if (long > short):
            strategy_name = 'ema_cross' + str(short) + '_' + str(long)
            short_description = 'EMA' + str(short)
            long_description = 'EMA' + str(long)

            signal_tmp = get_cross_signal(data[[short_description]].copy(), data[[long_description]].copy())
            bt_ema_tmp = signal_strategy(price_data.copy(), signal_tmp, strategy_name)
            bt_result_tmp = bt.run(bt_ema_tmp)

            print(strategy_name + ': ' + str(bt_result_tmp.stats.loc['total_return'].values[0]))

            if (total_return_best is None) or (bt_result_tmp.stats.loc['total_return'].values[0] > total_return_best):
                signal = signal_tmp
                bt_ema = bt_ema_tmp
                bt_result = bt_result_tmp
                short_value = short
                long_value =  long

                total_return_best = bt_result_tmp.stats.loc['total_return'].values[0]

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(20, 8))
fig.suptitle('EMA Cross ' + str(short_value) + '/' + str(long_value), fontsize=16)

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(bt_result.prices)
ax2.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
from strategy_utils import get_rsi_return_signal
from backtest_utils import signal_strategy

overbought_value = 70
oversold_value = 30

signal = get_rsi_return_signal(data[['RSI']].copy(), overbought_value=overbought_value, oversold_value=oversold_value)

bt_rsi_return = signal_strategy(price_data.copy(), signal, 'rsi_return')
bt_result = bt.run(bt_rsi_return)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [overbought_value] * len(data.index), label='overbought')
ax2.plot(data.index, [oversold_value] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
from strategy_utils import get_rsi_enter_signal
from backtest_utils import signal_strategy

overbought_value = 60
oversold_value = 40

signal = get_rsi_enter_signal(data[['RSI']].copy(), overbought_value=overbought_value, oversold_value=oversold_value)

bt_rsi_enter = signal_strategy(price_data.copy(), signal, 'rsi_enter')
bt_result = bt.run(bt_rsi_enter)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [overbought_value] * len(data.index), label='overbought')
ax2.plot(data.index, [oversold_value] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
from strategy_utils import get_inverted_rsi_signal
from backtest_utils import signal_strategy

overbought_value = 70
oversold_value = 30
signal = get_inverted_rsi_signal(data[['RSI']].copy(), overbought_value=overbought_value, oversold_value=oversold_value)

bt_inverted_rsi = signal_strategy(price_data.copy(), signal, 'rsi_inverted')
bt_result = bt.run(bt_inverted_rsi)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [overbought_value] * len(data.index), label='overbought')
ax2.plot(data.index, [oversold_value] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
###
from strategy_utils import get_dmi_signal
from backtest_utils import signal_strategy

signal = get_dmi_signal(data[['DI+']].copy(), data[['DI-']].copy(), data[['ADX']].copy())

bt_dmi = signal_strategy(price_data.copy(), signal, 'dmi')
bt_result = bt.run(bt_dmi)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, data[['DI+', 'DI-']])
ax2.plot(data.index, data[['ADX']], color='red')
ax2.plot(data.index, [25] * len(data.index), color = 'purple', linestyle='dashed')
ax2.set_ylabel('DMI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
from strategy_utils import get_rsi_adx_signal
from backtest_utils import signal_strategy

signal = get_rsi_adx_signal(data[['RSI']].copy(), data[['ADX']].copy(), data[['DI+']].copy(), data[['DI-']].copy(), overbought_value=70.0, oversold_value=30.0)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_rsi_adx = signal_strategy(price_data.copy(), signal, 'rsi_adx')

bt_result = bt.run(bt_rsi_adx)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_macd_signal
from backtest_utils import signal_strategy

signal = get_macd_signal(data[['MACDSignal']].copy(), data[['MACD']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=45)

plt.plot(combined_df['Price'])
plt.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
plt.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')

plt.legend()
plt.show()

In [ ]:
bt_macd = signal_strategy(price_data.copy(), signal, 'macd')

bt_result = bt.run(bt_macd)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_macd_rvi_signal
from backtest_utils import signal_strategy

signal = get_macd_rvi_signal(data[['MACDSignal']].copy(), data[['MACD']].copy(), data[['RVISignal']].copy(), data[['RVI']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_macd_rvi = signal_strategy(price_data.copy(), signal, 'macd_rvi')

bt_result = bt.run(bt_macd_rvi)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_sma_macd_signal
from backtest_utils import signal_strategy

signal = get_sma_macd_signal(data[['ClosePrice']].copy(), data[['SMA50']].copy(), data[['SMA100']].copy(), data[['MACD']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_sma_macd = signal_strategy(price_data.copy(), signal, 'sma_macd')

bt_result = bt.run(bt_sma_macd)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_adx_macd_signal
from backtest_utils import signal_strategy

signal = get_adx_macd_signal(data[['MACD']].copy(), data[['DI+']].copy(), data[['DI-']].copy(), data[['ADX']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_adx_macd = signal_strategy(price_data.copy(), signal, 'adx_macd')

bt_result = bt.run(bt_adx_macd)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_bbands_signal
from backtest_utils import signal_strategy

signal = get_bbands_signal(data[['ClosePrice']].copy(), data[['UpperBBand']].copy(), data[['LowerBBand']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_bbands = signal_strategy(price_data.copy(), signal, 'bbands')

bt_result = bt.run(bt_bbands)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_rsi_bbands_signal
from backtest_utils import signal_strategy

signal = get_rsi_bbands_signal(data[['ClosePrice']].copy(), data[['UpperBBand']].copy(), data[['LowerBBand']].copy(), data[['RSI_10']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
title = 'RSI + BBAnds Strategy ' + pair + '-' + strategy_interval

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data['ClosePrice'], label='Price')
ax2.plot(data['UpperBBand'], label='Upper band')
ax2.plot(data['MidiBBand'], label='Middle band')
ax2.plot(data['LowerBBand'], label='Lower band')
ax2.legend()
ax2.set_ylabel('Bollinger Bands')

ax3.plot(data.index, [75] * len(data.index), label='overbought')
ax3.plot(data.index, [20] * len(data.index), label='oversold')
ax3.plot(data.index, data['RSI_10'], label='RSI')
ax3.legend()
ax3.set_ylabel('RSI')

plt.xticks(rotation=45)
plt.show()

In [ ]:
bt_rsi_bbands = signal_strategy(price_data.copy(), signal, 'rsi_bbands')

bt_result = bt.run(bt_rsi_bbands)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_ema_atr_signal
from backtest_utils import signal_strategy

signal = get_ema_atr_signal(data[['ClosePrice']].copy(), data[['RSI']].copy(), data[['EMA200']].copy(), data[['ATR']].copy(), oversold_value=40)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
title = 'EMA + ATR Stop-loss Strategy ' + pair + '-' + strategy_interval

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.plot(data.index, data['EMA200'], label='EMA 200')
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [35] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

plt.xticks(rotation=45)
plt.show()

In [ ]:
bt_ema_atr = signal_strategy(price_data.copy(), signal, 'ema_atr')

bt_result = bt.run(bt_ema_atr)
print('Total Return: ', bt_result.stats.loc['total_return'].values[0])
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_rsi_atr_signal
from backtest_utils import signal_strategy

oversold_rsi_atr = 20
signal = get_rsi_atr_signal(data[['ClosePrice']].copy(), data[['RSI']].copy(), data[['ATR']].copy(), oversold_value=oversold_rsi_atr)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
title = 'RSI + ATR Stop-loss Strategy ' + pair + '-' + strategy_interval

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [oversold_rsi_atr] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

plt.xticks(rotation=45)
plt.show()

In [ ]:
bt_rsi_atr = signal_strategy(price_data.copy(), signal, 'rsi_atr')

bt_result = bt.run(bt_rsi_atr)
print('Total Return: ', bt_result.stats.loc['total_return'].values[0])
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from backtest_utils import signal_above_strategy, buy_and_hold_strategy

# Create signal strategy backtest
bt_sma5 = signal_above_strategy(price_data.copy(), data[['SMA5']].copy(), name='SMA5')
bt_sma10 = signal_above_strategy(price_data.copy(), data[['SMA10']].copy(), name='SMA10')
bt_sma20 = signal_above_strategy(price_data.copy(), data[['SMA20']].copy(), name='SMA20')
bt_sma30 = signal_above_strategy(price_data.copy(), data[['SMA30']].copy(), name='SMA30')
bt_sma50 = signal_above_strategy(price_data.copy(), data[['SMA50']].copy(), name='SMA50')
bt_sma100 = signal_above_strategy(price_data.copy(), data[['SMA100']].copy(), name='SMA100')
bt_sma200 = signal_above_strategy(price_data.copy(), data[['SMA200']].copy(), name='SMA200')

# Create benchmark strategy backtest
bt_benchmark = buy_and_hold_strategy(price_data.copy(), name='benchmark')

In [ ]:
# Run all backtests and plot the resutls
bt_results = bt.run(bt_sma5, bt_sma10, bt_sma20, bt_sma30, bt_sma50, bt_sma100, bt_sma200,
                    bt_sma, bt_ema, bt_macd, bt_sma_macd, bt_adx_macd, bt_macd_rvi, bt_dmi, bt_bbands,
                    #bt_rsi, bt_rsi_adx, bt_inverted_rsi, bt_rsi_return, bt_rsi_enter,
                    bt_benchmark)
# Plot results
bt_results.plot(title='Strategy benchmarking')
plt.show()

In [ ]:
bt_results.display()

In [ ]:
# Obtain all backtest stats
resInfo = bt_results.stats
ordered_strategies = resInfo.loc['total_return'].sort_values(ascending=False)
print(ordered_strategies)

best_strategy = ''
if resInfo.loc['total_return'].sort_values(ascending=False).index[0] == 'benchmark':
    best_strategy = resInfo.loc['total_return'].sort_values(ascending=False).index[1]
else:
    best_strategy = resInfo.loc['total_return'].sort_values(ascending=False).index[0]

bt_best = bt_results.backtests[best_strategy]

In [ ]:
bt_result_best = bt.run(bt_benchmark, bt_best)
bt_result_best.plot(title='Strategy benchmarking')
plt.show()

In [ ]:
transactions_best = bt_result_best.get_transactions(strategy_name=best_strategy)
transactions_best

In [ ]:
transactions_best.info()

In [ ]:
signal_best = transactions_best.copy()
signal_best = signal_best.reset_index()
signal_best = signal_best[['Date', 'quantity']]
signal_best.columns = ['OpenTime', 'signal']
signal_best['OpenTime'] = pd.to_datetime(signal_best['OpenTime'])
signal_best = signal_best.set_index('OpenTime')

signal_best[signal_best['signal'] < 0] = -1.0
signal_best[signal_best['signal'] > 0] = 1.0
signal_best = price_data.join(signal_best).fillna(0.0)

plt.figure(figsize=(20, 8))
plt.xticks(rotation=45)

plt.plot(signal_best['ClosePrice'])
plt.scatter(signal_best[signal_best['signal'] == 1.0].index, signal_best[signal_best['signal'] == 1.0]['ClosePrice'], label = 'Buy', marker='^', c='g')
plt.scatter(signal_best[signal_best['signal'] == -1.0].index, signal_best[signal_best['signal'] == -1.0]['ClosePrice'], label = 'Sell', marker='v', c='r')

plt.title('Price Chart & Historical Trades', fontweight="bold")
plt.legend()
plt.show()

In [ ]:
bt_result_ = bt.run(bt_benchmark, bt_sma10)
bt_result_.plot(title='Strategy benchmarking')
plt.show()

In [ ]:
bt_result_.get_transactions(strategy_name='SMA10')

In [ ]:
# Plot the backtest result
title = 'Backtest result ' + pair + '-' + strategy_interval
bt_results.plot(title=title)
plt.show()

# Get the lookback returns
lookback_returns = bt_results.display_lookback_returns()
print(lookback_returns)